**# Importanto o Dataset e verificando o conteúdo**

In [3]:
import sys
import os
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords') #carrego stopwrds para filtragem NLP

# Adiciona o caminho da pasta onde o arquivo 'dataset.py' está localizado
# Isso pula a necessidade de mencionar a pasta com hífen no import
caminho_raw = os.path.abspath(os.path.join('..', 'data', 'raw'))
if caminho_raw not in sys.path:
    sys.path.append(caminho_raw)

from dataset import carregar_dados

df = carregar_dados()
print(df.head())

[nltk_data] Downloading package stopwords to /home/rafael/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Baixando arquivo do Kaggle e movendo o dataset para pasta do projeto 


ConnectTimeout: HTTPSConnectionPool(host='api.kaggle.com', port=443): Max retries exceeded with url: /v1/datasets.DatasetApiService/GetDataset (Caused by ConnectTimeoutError(<HTTPSConnection(host='api.kaggle.com', port=443) at 0x71f5c5140190>, 'Connection to api.kaggle.com timed out. (connect timeout=None)'))

**Processamento e limpeza dos dados**


In [3]:
sys.path.append(os.path.abspath(os.path.join('..', 'data', 'raw')))
sys.path.append(os.path.abspath(os.path.join('..', 'data', 'processed')))


from dataset import verifica_vazios, verifica_frequentes
from data_processing import clean_text

df["clean_title"] = df["issue_title"].apply(clean_text)
df["clean_body"] = df["body"].apply(clean_text)


media_palavras_url = df["issue_url"].str.split().str.len().mean()
media_palavras_title = df["clean_title"].str.split().str.len().mean()
media_palavras_body = df["clean_body"].str.split().str.len().mean()

print(f"Média de palavras em url: {media_palavras_url}, em title: {media_palavras_title}, em body: {media_palavras_body}")

url_vazio = verifica_vazios(df,"issue_url")
titles_vazio = verifica_vazios(df,"clean_title")
body_vazio = verifica_vazios(df,"clean_body")

palavras_frequentes_titles = verifica_frequentes(df,"clean_title", 20)
print(f"Palavras mais frequentes em titles: {palavras_frequentes_titles}")
palavras_frequentes_body = verifica_frequentes(df,"clean_body", 20)
print(f"Palavras mais frequentes em body: {palavras_frequentes_body}")



Média de palavras em url: 1.0, em title: 5.459, em body: 28.4455
Resultados para 'issue_url':
- Valores Nulos (NaN): 0
- Textos Vazios/Espaços: 0
Resultados para 'clean_title':
- Valores Nulos (NaN): 0
- Textos Vazios/Espaços: 99
Resultados para 'clean_body':
- Valores Nulos (NaN): 0
- Textos Vazios/Espaços: 11
Palavras mais frequentes em titles: [('issu', 352), ('add', 151), ('bug', 137), ('found', 79), ('error', 49), ('test', 43), ('use', 39), ('api', 39), ('help', 34), ('fix', 34), ('new', 33), ('need', 26), ('log', 24), ('run', 23), ('list', 22), ('get', 20), ('set', 19), ('line', 17), ('typo', 17), ('task', 16)]
Palavras mais frequentes em body: [('close', 1244), ('add', 1102), ('issues', 927), ('columns', 714), ('list', 585), ('comment', 558), ('column', 542), ('trello', 529), ('update', 396), ('default', 387), ('settings', 378), ('via', 376), ('sync', 373), ('move', 369), ('custom', 367), ('board', 358), ('card', 357), ('attachments', 355), ('cards', 353), ('matisiekpl', 352)]



   **Análise descritiva dos dados processados**

- Média de palavras para entender quantos tokens irão ser processados pela LLM
    - Quantidade de células vazias em todas as colunas
        - não teve resultado de células vazias, todas preenchidas
    - Títulos - tamanhos médio de 5,4 palavras
    - Body - Aproximadamente 28 palavras
- Ideal para:
    - Embeddings
    - Chunking Leve
    - RAG Eficiente(baixo custo e boa semântica)